In [47]:
import requests
import pandas as pd
import json
import numpy as np
from datetime import datetime, date
import csv
import time
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
pd.options.mode.chained_assignment = None  # default='warn'

In [48]:
stock_pool_path = r"D:\Stock\pool\stock_pool.csv"
stock_pool_list = []
with open(stock_pool_path, newline='') as csvfile:
    rows = csv.reader(csvfile)
    row_idx = 0
    for row in rows:
        if row_idx == 0:
            title = row
        else:
            stock_pool_list.append(row)
        row_idx = row_idx + 1

In [49]:
target_csv_path = r"D:\Stock\daily_result\20240412_trend.csv"
daily_date = None
daily_data_list = []
with open(target_csv_path, newline='') as csvfile:
    rows = csv.reader(csvfile)
    row_idx = 0
    for row in rows:
        if row_idx == 0:
            daily_date = row
        elif row_idx == 1:
            title = row
        else:
            daily_data_list.append(row)
        row_idx = row_idx + 1

In [50]:
df_daily = pd.DataFrame(daily_data_list)
df_pool = pd.DataFrame(stock_pool_list)

In [51]:
new_pool_list = []
for i in range(len(df_daily)):
    daily_date_str = daily_date[0] + daily_date[1].zfill(2) + daily_date[2].zfill(2)
    id = df_daily.iloc[i, 0]
    name = df_daily.iloc[i, 1]
    pe = df_daily.iloc[i, 2]
    stock_yield = df_daily.iloc[i, 3]
    pb = df_daily.iloc[i, 4]
    price_suggest = df_daily.iloc[i, 5]
    isOTC = df_daily.iloc[i, 6]
    if not any(df_pool[1] == id):
        new_pool_list.append([daily_date_str, id, name, pe, stock_yield, pb, price_suggest, isOTC])

In [52]:
stock_pool_path = r"D:\Stock\pool\stock_pool.csv"
with open(stock_pool_path, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(new_pool_list)